# Generate data

In [13]:
import sys
sys.path.append("/Users/danmohad/Documents/Stanford/Courses/Year 5/CS230/project/nn_physical_concepts")

import scinet.ed_gaslaw as ed
from scinet import *

import numpy as np

In [22]:
# Dataset constants
name = "gaslaw"
l_obs = 3
l_ques = 2
l_ques_obs = 0
l_ans = 1

# Dataset parameters
n_ex = 1000
n_obs = 5
n_ques_obs = 0
n_ques = 1 # always
percent_validation = 5.0

p_r = [1.0, 2.0]
T_r = [3.0, 4.0]

# Network architecture parameters
latent_size = 1
encoder_layout = [150,100]
decoder_layout = [100,150]

In [23]:
N = n_ex
N_inp1 = l_obs*n_obs
N_inp2 = l_ques_obs*n_ques_obs + l_ques*n_ques
N_out = l_ans*n_ques

In [24]:
ed.generate_data(N, n_obs, p_interval=p_r, T_interval=T_r, species1="Ar", species2="He",
                 fileName=name + "_data");

In [25]:
td, vd, _, _, _ = dl.load(percent_validation, name + "_data")

In [26]:
net = nn.Network(N_inp1, latent_size, N_inp2, N_out, 
                 encoder_num_units=encoder_layout, decoder_num_units=decoder_layout,
                 name=name + "_net");

In [27]:
print(net.run(vd, net.recon_loss))

0.03108561


In [28]:
net.train(50, 256, 0.001, td, vd)
# check_epochs = 10
# net.train(check_epochs, 512, 0.001, td, vd, beta_fun=(lambda x: 0.001), test_step=10 )

  0%|          | 0/50 [00:00<?, ?it/s]

In [29]:
print(net.run(vd, net.recon_loss))

0.00030080706
